In [1]:
import os

import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "hawc.main.settings.local")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
from hawc.apps.assessment.models import Assessment
from hawc.apps.lit import models


In [3]:
assessment_id = 100500345

assessment = Assessment.objects.get(id=assessment_id)
references = assessment.references.all()

In [4]:
tag_pairs = models.Reference.objects.tag_pairs(references)
tag_pairs

[{'tag_id': 100513811, 'reference_id': 101398904},
 {'tag_id': 100513811, 'reference_id': 101398924},
 {'tag_id': 100513816, 'reference_id': 101396063},
 {'tag_id': 100513820, 'reference_id': 101396063},
 {'tag_id': 100513824, 'reference_id': 101396063},
 {'tag_id': 100513825, 'reference_id': 101396063},
 {'tag_id': 100513829, 'reference_id': 101396063},
 {'tag_id': 100513831, 'reference_id': 101396063},
 {'tag_id': 100513834, 'reference_id': 101396063},
 {'tag_id': 100513817, 'reference_id': 101396247},
 {'tag_id': 100513817, 'reference_id': 101396619},
 {'tag_id': 100513817, 'reference_id': 101397359},
 {'tag_id': 100513817, 'reference_id': 101398169},
 {'tag_id': 100513809, 'reference_id': 101396291},
 {'tag_id': 100513813, 'reference_id': 101396909},
 {'tag_id': 100513809, 'reference_id': 101398299},
 {'tag_id': 100513809, 'reference_id': 101395646},
 {'tag_id': 100513809, 'reference_id': 101396782},
 {'tag_id': 100513809, 'reference_id': 101395667},
 {'tag_id': 100513816, 'referen

In [5]:
search_pairs = list(models.Reference.searches.through.objects.filter(reference_id__in=references).values("search_id", "reference_id"))
search_pairs

[{'search_id': 100502921, 'reference_id': 101398904},
 {'search_id': 100502921, 'reference_id': 101398924},
 {'search_id': 100502914, 'reference_id': 101396063},
 {'search_id': 100502914, 'reference_id': 101396247},
 {'search_id': 100502915, 'reference_id': 101396619},
 {'search_id': 100502917, 'reference_id': 101397359},
 {'search_id': 100502919, 'reference_id': 101398169},
 {'search_id': 100502914, 'reference_id': 101396291},
 {'search_id': 100502916, 'reference_id': 101396909},
 {'search_id': 100502919, 'reference_id': 101398299},
 {'search_id': 100502913, 'reference_id': 101395646},
 {'search_id': 100502915, 'reference_id': 101396782},
 {'search_id': 100502913, 'reference_id': 101395667},
 {'search_id': 100502913, 'reference_id': 101395672},
 {'search_id': 100502921, 'reference_id': 101398733},
 {'search_id': 100502921, 'reference_id': 101398722},
 {'search_id': 100502919, 'reference_id': 101398487},
 {'search_id': 100502919, 'reference_id': 101398488},
 {'search_id': 100502919, 'r